In [1]:
#Annette Alca
#Advent of Code 2024
#Day 6

In [213]:
import time

In [214]:
#get input

fileN = 'Day6 input.txt'
file = open(fileN,'r')
lines = file.readlines()
file.close()
lines[:10]

['......#...........#..............................#..............................#...........................##...............#....\n',
 '.............#..............#........#......##...................................................#............##...#..........#...\n',
 '...#..................#..............#.........#...#.........................................................................#..#.\n',
 '.....#..........#...................#.....................#..........#...........#......#..#...................#..................\n',
 '...#...............................................##..................#............#.....#...................#...................\n',
 '.#.#..#...................#.##.................#..........#................................#......#.........##..................#.\n',
 '....................#...................#....#.........#....................#....................................................#\n',
 '..........#......#.....................

In [237]:
# get matrix

mtx = [] 

for l in lines:
    mtx.append(l.strip())

maxrow = len(mtx)
maxcol = len(mtx[0])

print(maxrow,maxcol)

130 130


In [238]:
#dictionary of the [next turn direction,row effect, col effect,limit]
turns = {'up':['right',-1,0,-1],
         'right':['down',0,1,maxcol],
         'down':['left',1,0,maxrow],
         'left':['up',0,-1,-1]}


def countSteps(row,col,direc):
    '''row, col are integer positions
    direc is direction: up, down, left, right
    returns 4 values: newrow,newcol,new direction, number of steps until next obstacle and new direction
    if end of matrix reached, new direction is "done" '''
    
    global mtx, maxcol, maxrow, turns
    
    assert direc in ['up', 'right', 'down', 'left'], 'Check direction and reenter please.'
    
    steps = 0
    newdirec = turns[direc][0]
    rowchange = turns[direc][1]
    colchange = turns[direc][2]
    limit = turns[direc][3]
    
    if colchange==0: #same column, direc is up or down
        for newrow in range(row+rowchange,limit,rowchange):
            new = mtx[newrow][col]
            if new == '.': #replacing '.' with 'X' below, adding 1 position
                rowstring=mtx[newrow][:col]+'X'+mtx[newrow][col+1:] 
                mtx[newrow]=rowstring
                steps +=1

            elif new == '#':  #stop and turn
                return [newrow-rowchange,col,newdirec,steps] #if block encountered
 
        return [newrow,col,'done',steps] #if out of matrix
    
    else: #same row, direc is left or right
        for newcol in range(col+colchange,limit,colchange):
            new = mtx[row][newcol]
            if new == '.':  #replacing '.' with 'X' below, adding 1 position
                rowstring=mtx[row][:newcol]+'X'+mtx[row][newcol+1:] 
                mtx[row]=rowstring
                steps +=1

            elif new == '#': #stop and turn
                return [row,newcol-colchange,newdirec,steps] #if block encountered

        return [row,newcol,'done',steps] #if out of matrix
    
        

In [239]:
#Part 1
start = time.time()

totalsteps = 1
totalturns = 0

#find coordinates of ^ for start

for m in mtx:
    if '^' in m:
        row = mtx.index(m)
        col = m.index('^')
        direc = 'up'
        #print('orig:',row,col,'0',totalsteps)
        #print(direc,end=': ')


#tracking, counting totalsteps        
        
while direc != 'done':
    totalturns +=1
    row,col,direc,steps = countTurn(row,col,direc)
    totalsteps += steps
    #print(row,col,steps,totalsteps)
    #print(direc,end=': ')
    
print(totalsteps)
print("--- %s seconds ---" % (time.time() - start))


exited matrix
4454
--- 0.006059885025024414 seconds ---


In [235]:
#number of turns before exiting
totalturns

133

In [240]:
#generate original sample matrix

dtx = mtx.copy() # where all 'X' marked

origmtx = [] #generate original mtx with no 'X', just '.'
for t in dtx:
    origmtx.append(t.replace('X','.'))
origmtx[:5]


['......#...........#..............................#..............................#...........................##...............#....',
 '.............#..............#........#......##...................................................#............##...#..........#...',
 '...#..................#..............#.........#...#.........................................................................#..#.',
 '.....#..........#...................#.....................#..........#...........#......#..#...................#..................',
 '...#...............................................##..................#............#.....#...................#...................']

In [241]:
#Part2
#New obstacles can only be in travelled path, spots marked with 'X' in dtx
#get matrix positions with X

Xspots = []

for row in range(maxrow):
    if 'X' in dtx[row]:
        for col in range(maxcol):
            if dtx[row][col] == 'X':
                Xspots.append([row,col])
                
len(Xspots)

4453

In [246]:
#Part2


#timing
start = time.time()

#find starting point
for m in mtx:
    if '^' in m:
        rowstart = mtx.index(m)
        colstart = m.index('^')
        direcstart = 'up'
        print('Starting point:',rowstart,colstart,direcstart)
        #print(direc)


#tracking, counting totalsteps       

maxturns = 200 #original solution max 133 turns, so 200 is generous 

iloop_coords =  [] #list of coordinates where obstacle can be placed to create infinite loop for guard


for coords in Xspots:
    zerosteps=0
    turns2=0
    
    #restore starting conditions
    row = rowstart
    col = colstart
    direc = direcstart
    
    #create mtx with new obstacle
    mtx = origmtx.copy()
    testrow = coords[0]
    testcol = coords [1]
    rowstring=mtx[testrow][:testcol]+'#'+mtx[testrow][testcol+1:] #place obstacle at coordinates  
    mtx[testrow]=rowstring #redo mtx with new obstacle
    
    while turns2<=maxturns and direc!='done' and zerosteps<15:
        turns2 +=1
        row,col,direc,steps = countSteps(row,col,direc)
        if steps==0:
            zerosteps +=1
        else:
            zerosteps = 0
        if zerosteps>14 and coords not in iloop_coords: #the guard is now in a loop taking 14 turns and tracking spots they have been to
            iloop_coords.append(coords)
            continue
    #print(coords,zerosteps,turns2)
    #print(mtx)

print('Part2 Answer = ',len(iloop_coords))
print("--- %s seconds ---" % (time.time() - start))

Starting point: 45 42 up
Part2 Answer =  1503
--- 4.691971063613892 seconds ---
